In [1]:
# The path to the local git repo for Indic NLP library
INDIC_NLP_LIB_HOME=r"/Users/mayuresh/Desktop/indic_nlp_library"

# The path to the local git repo for Indic NLP Resources
INDIC_NLP_RESOURCES=r"/Users/mayuresh/Desktop/indic_nlp_resources"

import pandas as pd
import numpy as np
import warnings
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)
from indicnlp import loader
loader.load()


In [2]:
training_dataset = pd.read_csv("/Users/mayuresh/Desktop/MOLD/Mold/MOLD_Training2.csv")
training_dataset.head()
training_dataset.dropna()
training_dataset['subtask_c'].fillna("NULL")

0       GRP
1       GRP
2       GRP
3       GRP
4       GRP
       ... 
2493    OTH
2494    OTH
2495    OTH
2496    OTH
2497    OTH
Name: subtask_c, Length: 2498, dtype: object

In [3]:
tweets = training_dataset["tweet"]
tweets

0       राम कदम वागण्यात नाही तर बोलण्यात चुकला  बीजेप...
1       हीच का तुमची शिवसेने चि शिकवण... आपली आई म्हणज...
2       हे वाचा गाढवांनो. आणि हे ही सांगा की तुमच्या म...
3              भक्त आंधळे असतात. मूर्खा ना काही कळत नाही.
4       १९६६ साली छत्रपती शिवाजी महाराज या जागतिक दर्ज...
                              ...                        
2493     पण बरेच जण जन्मजात मंद   मूर्ख आणि अंधभक्त अस...
2494     कर्नाटक मध्ये जेव्हा छत्रपती शिवाजी महाराजांच...
2495     मंद...सत्तेचे लालची तुम्ही म्हणून आज गुंडांना...
2496                           आयटी सेल वाले दळभद्री 🤣🤣🤣 
2497     मविआ चे चाटे काय गोट्या चोळत बसले काय नेत्यां...
Name: tweet, Length: 2498, dtype: object

In [4]:
level_A = training_dataset[["subtask_a"]]
level_B = training_dataset.query("subtask_a == 'Offensive'")[["subtask_b"]]
level_C = training_dataset.query("subtask_b == 'TIN'")[["subtask_c"]]

In [5]:
from indicnlp.tokenize import indic_tokenize  


In [6]:
stopwords_file = open("/Users/mayuresh/Desktop/MOLD/Mold/stopwords.txt")
stopwords = stopwords_file.read().splitlines()
print(stopwords)

['अधिक ', 'अनेक ', 'अशी ', 'असलयाचे', 'असलेल्या', 'असा', 'असून', 'असे', 'आज', 'आणि', 'आता', 'आपल्या', 'आला', 'आली', 'आले', 'आहे', 'आहेत', 'एक', 'एका', 'कमी', 'करणयात', 'करून', 'का', 'काम', 'काय', 'काही', 'किवा', 'की', 'केला', 'केली', 'केले', 'कोटी', 'गेल्या', 'घेऊन', 'जात', 'झाला', 'झाली', 'झाले', 'झालेल्या', 'टा', 'डॉ', 'तर', 'तरी', 'तसेच', 'ता', 'ती', 'तीन', 'ते', 'तो', 'त्या', 'त्याचा', 'त्याची', 'त्याच्या', 'त्याना', 'त्यानी', 'त्यामुळे', 'त्री', 'दिली', 'दोन', 'न', 'नाही', 'निर्ण्य', 'पण', 'पम', 'परयतन', 'पाटील', 'म', 'मात्र', 'माहिती', 'मी', 'मुबी', 'म्हणजे', 'म्हणाले', 'म्हणून', 'या', 'याचा', 'याची', 'याच्या', 'याना', 'यानी', 'येणार', 'येत', 'येथील', 'येथे', 'लाख', 'व', 'व्यकत', 'सर्व', 'सागित्ले', 'सुरू', 'हजार', 'हा', 'ही', 'हे', 'होणार', 'होत', 'होता', 'होती', 'होते']


In [17]:
def clean(tweet):
    removal_list = ['URL','\'ve','n\'t','\'s','\'m','!']
    for element in removal_list:
        tweet = str(tweet).replace(element,'')
    
    return tweet

In [18]:
# for tweet in tweets:
#     tweet = remove_noise(str(tweet))
iterator_map = map(clean,tweets)
tweets = list(iterator_map)

In [19]:
print(tweets)

['राम कदम वागण्यात नाही तर बोलण्यात चुकला  बीजेपी मग अजितदादा कुठे तुमच्या तोंडात मूतले होते का?', 'हीच का तुमची शिवसेने चि शिकवण... आपली आई म्हणजे दैवत आणि दुसऱ्याची आई म्हणजे रांड... 🤬🤬', 'हे वाचा गाढवांनो. आणि हे ही सांगा की तुमच्या मॅडम आणि पप्पूने 2008 साली काय करार केला चीनी कम्युनि…', 'भक्त आंधळे असतात. मूर्खा ना काही कळत नाही.', '१९६६ साली छत्रपती शिवाजी महाराज या जागतिक दर्जाच्या सम्राटाचे नाव (brand) वापरून ज्यांनी  शिव सेना स्थापन केली त्यांची…', 'भक्तांची आधुनिक झासी कंगना रानावत हिचे झाशीच्या राणी बद्दलच वक्तव्य बघा   तुम्हाला लाज वाटायला पाहिजे   दळभद्री बिंडोक भ…', 'शांत राहिले की ही बाळासाहेबांची सेना नाही अन् थोडे अशांत झाले की आतंकी सेना.. त्यांच्या मतावर तेच ठाम नाहीत. मंद भक्त', 'यांना पोर्टस्टार म्हणणाऱ्या नालायक जातीच्या ला महाराष्ट्रातील विशिष्ट पक्षाचे कार्यक…', 'मरणार.. लाचारी रक्तात भिणली आहे यांच्या. आरश्यासमोर उभे राहू शकत नाहीत म्हणून त्यांची दळभद्री तोंडे घेऊन आपल्या समोर फडकत असतात.', 'अरे येड्याहो तुम्ही तर मर्चंट नेव्हीतल्या माणसाला भारतीय नौदलाचा अधिका